### Source notebook is loaded

All the libraries and formulas needed for running the ANN-GARCH model are loaded. Please install all the libraries before running the code.

In [3]:
%run Source/1_ANN_ARCH.ipynb

### ANN-GARH model

Model run for obtaining the test error of the ANN-GARCH model with the optimum dropout level obtained with the train set. The forecasts and VaR values of the different models are saved in a csv file.

In [4]:
#Index of end dates, database for validation and dataframe to collect the results are created. Model variables are defined.
Start='2016-01-09'; End='2021-01-08'; IndexEndDays=yf.download("^GSPC",start=Start,  end=End, progress=False).index
Lag=1; LagSD=5; Dropout=0.15; LearningRate=0.01; Epochs=5000; Alpha=0.005; DF=3.28
DataValidation = DatabaseGeneration(yf.download("^GSPC",start='2000-01-01', end=date.today()+timedelta(days=1), progress=False), Lag, LagSD)
ResultsCollection=pd.DataFrame({'Date_Forecast': [], 'Forecast_ANN_ARCH': [],'Forecast_GARCH':[],'Forecast_GJR_GARCH':[], 'Forecast_TARCH':[],'Forecast_EGARCH':[],'Forecast_AVGARCH':[],'Forecast_FIGARCH':[],'ReturnForecast':[],'TrueSD':[], 'VaR_ANN_ARCH':[], 'VaR_GARCH':[], 'VaR_GJR_GARCH':[], 'VaR_TARCH':[], 'VaR_EGARCH':[], 'VaR_AVGARCH':[], 'VaR_FIGARCH':[]})
#Loop for generating the results
for i in tqdm(range(IndexEndDays.shape[0])):
    #Database is downloaded from yahoo finance and lag of returns defined
    Database=yf.download("^GSPC",start=IndexEndDays[i].date()-timedelta(days=650), end=IndexEndDays[i].date(), progress=False)
    #Database for fitting the models is generated
    Data = DatabaseGeneration (Database, Lag, LagSD)
    #Fitting of ANN-ARCH model, ARCH models and forecasting of the next volatility value
    ANN_ARCH_Model = ANN_ARCH_Fit (Data, Lag, LagSD, Dropout, LearningRate, Epochs, Alpha, DF)
    #VaR of ARCH models is computed
    VaR_ARCH_Models=VaR_AR_Total(Alpha, ANN_ARCH_Model['GARCH_fit'], ANN_ARCH_Model['GJR_GARCH_fit'], ANN_ARCH_Model['TARCH_fit'], ANN_ARCH_Model['EGARCH_fit'], ANN_ARCH_Model['AVGARCH_fit'], ANN_ARCH_Model['FIGARCH_fit'],ANN_ARCH_Model['GARCH'], ANN_ARCH_Model['GJR_GARCH'], ANN_ARCH_Model['TARCH'], ANN_ARCH_Model['EGARCH'], ANN_ARCH_Model['AVGARCH'], ANN_ARCH_Model['FIGARCH'])
    #Results are collected
    IterResults={'Date_Forecast': ANN_ARCH_Model['Date_Forecast'].date(), 'Forecast_ANN_ARCH': ANN_ARCH_Model['Forecast_ANN_ARCH'],'Forecast_GARCH':ANN_ARCH_Model['Forecast_GARCH']/100,'Forecast_GJR_GARCH':ANN_ARCH_Model['Forecast_GJR_GARCH']/100, 'Forecast_TARCH':ANN_ARCH_Model['Forecast_TARCH']/100,'Forecast_EGARCH':ANN_ARCH_Model['Forecast_EGARCH']/100,'Forecast_AVGARCH':ANN_ARCH_Model['Forecast_AVGARCH']/100,'Forecast_FIGARCH':ANN_ARCH_Model['Forecast_FIGARCH']/100,'ReturnForecast':ANN_ARCH_Model['ReturnForecast'],'TrueSD':DataValidation[DataValidation.index==pd.to_datetime(ANN_ARCH_Model['Date_Forecast'].date())]['TrueSD'][0], 'VaR_ANN_ARCH': ANN_ARCH_Model['VaR'], 'VaR_GARCH':VaR_ARCH_Models['VaR_GARCH'][0][0], 'VaR_GJR_GARCH':VaR_ARCH_Models['VaR_GJR_GARCH'][0][0], 'VaR_TARCH':VaR_ARCH_Models['VaR_TARCH'][0][0], 'VaR_EGARCH':VaR_ARCH_Models['VaR_EGARCH'][0][0], 'VaR_AVGARCH':VaR_ARCH_Models['VaR_AVGARCH'][0][0], 'VaR_FIGARCH':VaR_ARCH_Models['VaR_FIGARCH'][0][0]}
    ResultsCollection=ResultsCollection.append(IterResults, ignore_index=True)
    #Results are saved
    ResultsCollection.to_csv('Results/1_ANN_ARCH.csv',index=False)

100%|██████████| 1259/1259 [17:02:02<00:00, 48.71s/it]  
